# YouTube Simulation (in notebook form)

### Initializing environment packages and variables 📚

In [2]:
# Importing libraries/packages
import random
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Importing class files
from agent import Agent
from video import Video
from activity import Activity
from behavior_reference import BEHAVIOR_ARCHETYPE_PARAMETERS
from behavior_reference import AGENT_ARCHETYPE_DISTRIBUTION

In [3]:
# Defining our macro values to be referenced later on
NUM_VIDEOS = 1000;
NUM_AGENTS = 100;

### Creation of Video Objects 🎥

In [4]:

# Generate random view count (views range from 1 to 1 million)
random_view_counts = []
for i in range(NUM_VIDEOS):
    r = random.randint(1, 1000000)
    random_view_counts.append(r)


# Generate random (unique) video ids
# resultant random numbers list
random_video_ids = []
# traversing the loop 1000 times
for i in range(NUM_VIDEOS):

    # r=random.randint(1,100000)
    # # checking whether the generated random number is not in the
    # # randomList
    # if r not in random_video_ids:
    #     # appending the random number to the resultant list, if the condition is true
        random_video_ids.append(i)


# Generate random video length
random_vid_lengths = []
for i in range(NUM_VIDEOS):
    r = random.randint(1, 100)
    random_vid_lengths.append(r)

# Generate random video extremeness
random_extremeness = []
for i in range(NUM_VIDEOS):
    r = random.randint(0*0,1*10)/10
    random_extremeness.append(r)


# Generate random number of thumbs up
random_thumbs_up = []
for i in range(NUM_VIDEOS):
    r = random.randint(0, 50000)
    random_thumbs_up.append(r)


# Use the above arrays to create video objects
all_videos = []
for i in range(NUM_VIDEOS):   # create 1000 videos


    # Grab the data points for the video
    views = random_view_counts[i];
    vid_id = random_video_ids[i];
    length = random_vid_lengths[i];
    extremeness = random_extremeness[i];
    thumbs_up = random_thumbs_up[i];

    # Create the video object
    random_vid = Video(views, vid_id, length, extremeness, thumbs_up);

    # Add the video object to our array of videos
    all_videos.append(random_vid);

In the cell below, we can test that the video objects are created properly.

In [5]:
# Let's make sure this worked.

print("Information for the first three videos:");
for i in range(3):
    print("Views: " + str(all_videos[i].views));
    print("Video ID: " + str(all_videos[i].vid_id));
    # print("Length: " + str(all_videos[i].length) +  " minutes");
    print("Extremeness: " + str(all_videos[i].extremeness));
    # print("Thumbs up count: " + str(all_videos[i].thumbs_up));
    # print("");

# # Let's test the watch function
for i in range(10):
    Activity.watch(all_videos[i])

Information for the first three videos:
Views: 714564
Video ID: 0
Extremeness: 0.9
Views: 736835
Video ID: 1
Extremeness: 0.5
Views: 722733
Video ID: 2
Extremeness: 0.2
No thumbs up given
No thumbs up given
No thumbs up given
Thumbs up given
Thumbs up given
Thumbs up given
No thumbs up given
No thumbs up given
Thumbs up given
Thumbs up given


### Creating our Agents 👩‍🔬

In [6]:
# Now that we've generated the videos, we can generate our agents.
# Let's start with 100 agents.
# This means we will have the following archetype counts:
"""
AGENT_ARCHETYPE_DISTRIBUTION = {
    "progressive_activist": 8,
    "traditional_liberal": 11,
    "passive_liberal": 15,
    "politically_disengaged": 26,
    "moderate": 15,
    "traditional_conservative": 19,
    "devoted_conservative": 6
}    
"""
# So, 8 progressive activists, 11 traditional liberals, 15 passive liberals, 26 politically disengaged,
# 15 moderates, 19 traditional conservatives, and 6 devoted conservatives.
# Since we have 100 agents, their IDs can just be 1-100 in the order they are created.

# Creating an array to hold our agents
our_agents = [];

id_counter = 0;
# Generate the progressive activists (8)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["progressive_activist"]):    
    our_agent = Agent(False, "progressive_activist", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

# Generate the traditional liberals (11)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["traditional_liberal"]):
    our_agent = Agent(False, "traditional_liberal", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

# Generate the passive liberals (15)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["passive_liberal"]):
    our_agent = Agent(False, "passive_liberal", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

# Generate the politically disengaged (26)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["politically_disengaged"]):
    our_agent = Agent(False, "politically_disengaged", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

# Generate the moderates (15)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["moderate"]):
    our_agent = Agent(False, "moderate", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

# Generate the traditional conservatives (19)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["traditional_conservative"]):
    our_agent = Agent(False, "traditional_conservative", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

# Generate the devoted conservatives (6)
for i in range(AGENT_ARCHETYPE_DISTRIBUTION["devoted_conservative"]):
    our_agent = Agent(False, "devoted_conservative", id_counter, "");
    our_agents.append(our_agent);
    id_counter += 1;

Once again, we can grab some data to test that these agents were created successfully.

In [7]:
# Now, let's see if this actually works
for i in range(NUM_AGENTS):
    print("Agent ID: " + str(our_agents[i].agent_id) + "\tArchetype: " + our_agents[i].archetype);

Agent ID: 0	Archetype: progressive_activist
Agent ID: 1	Archetype: progressive_activist
Agent ID: 2	Archetype: progressive_activist
Agent ID: 3	Archetype: progressive_activist
Agent ID: 4	Archetype: progressive_activist
Agent ID: 5	Archetype: progressive_activist
Agent ID: 6	Archetype: progressive_activist
Agent ID: 7	Archetype: progressive_activist
Agent ID: 8	Archetype: traditional_liberal
Agent ID: 9	Archetype: traditional_liberal
Agent ID: 10	Archetype: traditional_liberal
Agent ID: 11	Archetype: traditional_liberal
Agent ID: 12	Archetype: traditional_liberal
Agent ID: 13	Archetype: traditional_liberal
Agent ID: 14	Archetype: traditional_liberal
Agent ID: 15	Archetype: traditional_liberal
Agent ID: 16	Archetype: traditional_liberal
Agent ID: 17	Archetype: traditional_liberal
Agent ID: 18	Archetype: traditional_liberal
Agent ID: 19	Archetype: passive_liberal
Agent ID: 20	Archetype: passive_liberal
Agent ID: 21	Archetype: passive_liberal
Agent ID: 22	Archetype: passive_liberal
Agent 

### Simulating a Day 📅

So, what does an agent do in a given day?

* Click on a video (they will be provided with a random video at start of day)
    * Decide to watch the video if it aligns with their archetype parameters 
        * (todo: figure out how to code that. Probably just a bunch of checks for each archetype.)
    * Actually watch the video
    * Increment the video views
    * Add the video length to their total time spent watching for today
    * Flip a coin to determine if a thumbs up is left
    * If total time spent watching for today is under their archetype's daily limit, find a new video.
        * If over time, stop watching and end the day.


In [8]:
# Function to click on a video (agent will be provided with a random video at start of day)
def suggest_video(video_list, num_videos):
    random_video_id = random.randrange(num_videos);
    return video_list[random_video_id];
    

# Function to print the attributes of a video object
def display_vid_attrs(our_video):
    rand_vid_views = our_video.views
    rand_vid_id = our_video.vid_id
    rand_vid_length = our_video.length
    rand_vid_extremeness = our_video.extremeness
    rand_vid_thumbsup = our_video.thumbs_up

    print("Video ID: " + str(rand_vid_id))
    print("View Count: " + str(rand_vid_views))
    print("Length: " + str(rand_vid_length) + " minutes")
    print("Extremeness: " + str(rand_vid_extremeness))
    print("Thumbs Up Count: " + str(rand_vid_thumbsup))


# Function to display an agent's ID and archetype
def display_agent(our_agent):
    print("Agent ID: " + str(our_agent.agent_id));
    print("Archetype: " + our_agent.archetype)

In [9]:
# Test the video clicking
suggested_video = suggest_video(all_videos, NUM_VIDEOS)
display_vid_attrs(suggested_video)   

Video ID: 287
View Count: 298299
Length: 15 minutes
Extremeness: 0.8
Thumbs Up Count: 30408


In [18]:
# Decide to watch the video or not, then actually watch it.

# First, we must grab our agent.
# For the sake of my sanity in coding this right now, we're just going to start with the first agent in the list.
# Eventually, this will happen for every agent.

# Oh God. I have to go back and add the actual attributes? Maybe????? For the archetype to each agent.
# OR can i just access their info based on archetype from behavior_reference.py????? Can I try that?
# YES! We can use nested dictionaries

daily_agent = our_agents[0]


daily_agent_archetype = daily_agent.archetype

# These two need to be declared OUTSIDE of the run for each video.
# So, declare them within the day for a given agent, but OUTSIDE of the actual video selection checking loop.
# Otherwise, they don't actually get updated each time.
minutes_watched_today = 0   # how many minutes the agent has watched today
vids_watched_today = 0  # how many videos the agent watched today

activity_log = []  # ids of the videos the agent watched today


# Get the values for our agent's archetype

daily_agent_longest_vid = BEHAVIOR_ARCHETYPE_PARAMETERS[daily_agent_archetype]["longest_vid_threshold"]
daily_agent_yt_threshold = BEHAVIOR_ARCHETYPE_PARAMETERS[daily_agent_archetype]["yt_time_threshold"]
daily_agent_pol_aff = BEHAVIOR_ARCHETYPE_PARAMETERS[daily_agent_archetype]["political_affiliation"]
daily_agent_vid_extr = BEHAVIOR_ARCHETYPE_PARAMETERS[daily_agent_archetype]["video_extremity"]
daily_agent_pop_thresh = BEHAVIOR_ARCHETYPE_PARAMETERS[daily_agent_archetype]["popularity_threshold"]


# Now, let's pair this agent with a random video.
# Can we write a function that does all these checks?

time_left_check = True; # means we have enough time for the agent to keep watching videos

while(time_left_check == True):

    suggested_video = suggest_video(all_videos, NUM_VIDEOS)

    display_agent(daily_agent)
    print("")
    display_vid_attrs(suggested_video)


    # Compare our agent's thresholds to the attributes of the video

    print("")

    # Check minimum view threshold
    if(suggested_video.views >= daily_agent_pop_thresh):
        popularity_check = True
        print("Video is popular enough.")
    else:
        popularity_check = False
        print("Video is not popular enough.")

    # Check agent's max viewing length
    if(suggested_video.length < daily_agent_longest_vid):
        length_check = True
        print("Video is proper length.")
    else:
        length_check = False
        print("Video is too long.")

    # Check if watching this video would exceed the agent's daily threshold
    potential_mins_watched = minutes_watched_today + suggested_video.length
    if(potential_mins_watched < daily_agent_yt_threshold):
        time_left_check = True
        print("Still time to watch this video.")
    else:
        time_left_check = False
        print("Not enough time left to watch this video.")

    # Check if this video is too extreme for the agent.

    # Left-leaning archetypes will watch anything at 0.5 and above. Right-leaning will watch 0.5 and below.
    if(daily_agent_pol_aff == "left"):
        # Will not watch anything under 0.5 extremeness
        # If video extremeness is < 0.5 or higher than their extremeness value, do not watch.
        if((suggested_video.extremeness < 0.5) or (suggested_video.extremeness > daily_agent_vid_extr)):
            extreme_check = False
            print("Video was too extreme.")
        else:
            extreme_check = True
            print("Video is within extremeness bounds (between 0.5 and agent's archetype value).")
    elif(daily_agent_pol_aff == "right"):
        # Will not watch anything amove 0.5 extremeness
        # If video extremeness is > 0.5 or lower than their extremeness value (0.0 is extreme here), do not watch.
        if((suggested_video.extremeness > 0.5) or (suggested_video.extremeness < daily_agent_vid_extr)):
            extreme_check = False
            print("Video was too extreme.")
        else:
            extreme_check = True
            print("Video is within extremeness bounds (between agent's archetype value and 0.5).")
    elif(daily_agent_pol_aff == "middle"):
        #todo: figure out what goes here
        extreme_check = True # this is PLACEHOLDER please eventually delete this


    # If all four checks pass, congrats! The agent will watch the video.
    if(popularity_check and length_check and time_left_check and extreme_check):
        print("Success! Agent #" + str(daily_agent.agent_id) + " will be watching video #" + str(suggested_video.vid_id) + ".")
        Activity.watch(suggested_video)     # Agent actually watches the video.
        minutes_watched_today = minutes_watched_today + suggested_video.length
        vids_watched_today = vids_watched_today + 1
        print("\nTotal minutes watched today is now " + str(minutes_watched_today) + ".")


print("\nVideos watched today: " + str(vids_watched_today))
print("Minutes watched today: " + str(minutes_watched_today))



# TODO for next time:
# All of the code in this cell should be encapsulated into a function.
# That way, you can run a for loop to iterate through each agent in the list, then feed that agent in as a parameter to the function.
# That for loop constitutes one day of YouTube.
# This cell also only covers one pass of videos for an agent; you need to utilize the time_left_check to see how much time is left after each video.
# If there is time left, run everything agian.
# OH! Maybe do a while loop where while(time_left_check < daily_agent_yt_threshold), the whole getting a video and all the other checks thing happens.
# Before this can be executed with other agents, however, you need to figure out extremeness for the middle ground agents.
# Maybe if the extremeness is within 0.2 of the agent's value, the video can be watched. Idk.

# Also, when you hopefully eventually get this whole thing working, use the Jupyter format to add lots of background and descriptive text.





Agent ID: 0
Archetype: progressive_activist

Video ID: 2
View Count: 722734
Length: 100 minutes
Extremeness: 0.2
Thumbs Up Count: 37775

Video is popular enough.
Video is too long.
Still time to watch this video.
Video was too extreme.
Agent ID: 0
Archetype: progressive_activist

Video ID: 246
View Count: 131146
Length: 4 minutes
Extremeness: 0.7
Thumbs Up Count: 10229

Video is popular enough.
Video is proper length.
Still time to watch this video.
Video is within extremeness bounds (between 0.5 and agent's archetype value).
Success! Agent #0 will be watching video #246.
Thumbs up given

Total minutes watched today is now 4.
Agent ID: 0
Archetype: progressive_activist

Video ID: 630
View Count: 718344
Length: 43 minutes
Extremeness: 0.8
Thumbs Up Count: 25288

Video is popular enough.
Video is proper length.
Still time to watch this video.
Video is within extremeness bounds (between 0.5 and agent's archetype value).
Success! Agent #0 will be watching video #630.
No thumbs up given

To